# Desenvolver o codigo para automatizar a busca no Rec Aq

* https://www.reclameaqui.com.br/empresa/bet365/lista-reclamacoes/?pagina=1

In [1]:
from bs4 import BeautifulSoup
from unidecode import unidecode

import pandas as pd
import numpy as np

import string
import requests

In [2]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}

def get_complains(company_id, company_name):

    complains = {'company_name': [], 'description': [], 'href': []}
    
    for n in range(0, 1001, 10):
        url = f'https://iosearch.reclameaqui.com.br/raichu-io-site-search-v1/query/companyComplains/10/{n}'
        res = requests.get(url, params={'company': company_id}, headers=HEADERS).json()

        for x in range(10):
            data = res['complainResult']['complains']['data']
            
            description = data[x]['description']
            soup = BeautifulSoup(description, 'lxml').text
            complains['description'].append(soup)
            
            id_ = data[x]['id']
            title = data[x]['title']
            clean_title = title.translate(str.maketrans('', '', string.punctuation))
            href = clean_title.lower().strip() + '_' + id_
            href = unidecode(href.replace(' ', '-'))
            complains['href'].append(href)
            
            complains['company_name'].append(company_name)
    
    return complains

In [3]:
data = get_complains(76804, 'bet365')

/home/vitor/.pyenv/versions/3.10.6/envs/estudo/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [4]:
df = pd.DataFrame(data)
df

,company_name,description,href
0,bet365,Fiz uma aposta e estava ganhando faltando apen...,minha-aposta-foi-manipulada-pelo-sistema_U8v16...
1,bet365,"Fiz uma conta , fiz o depósito via pix, comece...",reembolso_EKRMsEKk_rbmC5IZ
2,bet365,"Eu criei uma conta nova recentemente, usei ach...",conta-nova-restricao_ggudKvG7nIqFoj3L
3,bet365,Fiz uma aposta Vasco vencedor no segundo tempo...,solucao-de-aposta-do-jogo-vasco-e-guarani_mCK6...
4,bet365,Prezados minha conta foi bloqueada e não consi...,nao-consigo-entrar-na-minha-conta_37a8Qrk-B1kW...
...,...,...,...
1005,bet365,Quero meus 34.21E meu bonus 34.21,minha-aposta-foi-encerrada-sem-minha-autorizac...
1006,bet365,Fizeram um pix que eu ganhei de 100 reais no m...,devolucao-de-pix_iSqtJj8BWVfrRne3
1007,bet365,"O acesso à minha conta foi cortado, 50 reais f...",engoliram-r5000-do-meu-deposito-cortaram-o-ace...
1008,bet365,Estou a 2 semanas tentando fazer deposito na c...,deposito-malsucedido_9ss-d9wwdVJnallQ


In [11]:
%%time

for i, company_name, href in zip(df.index, df.company_name, df.href):
    try:
        url = f"https://www.reclameaqui.com.br/{company_name}/{href}/"
        res = requests.get(url, headers=HEADERS)

        soup = BeautifulSoup(res.text, 'lxml')
        complain = soup.find('p', {'class': 'lzlu7c-17 fXwQIB'}).text
        df.iloc[i]['full_complain'] = complain
    except:
        print(href)
        continue

df.head()

nao-consigo-receber-meu-dinheiro_alI6h__ymn6ELXug
deposito-incompleto_IBMyEYzVy7lLkvxC
CPU times: user 1min 48s, sys: 877 ms, total: 1min 49s
Wall time: 20min 23s


,company_name,description,href
0,bet365,Fiz uma aposta e estava ganhando faltando apen...,minha-aposta-foi-manipulada-pelo-sistema_U8v16...
1,bet365,"Fiz uma conta , fiz o depósito via pix, comece...",reembolso_EKRMsEKk_rbmC5IZ
2,bet365,"Eu criei uma conta nova recentemente, usei ach...",conta-nova-restricao_ggudKvG7nIqFoj3L
3,bet365,Fiz uma aposta Vasco vencedor no segundo tempo...,solucao-de-aposta-do-jogo-vasco-e-guarani_mCK6...
4,bet365,Prezados minha conta foi bloqueada e não consi...,nao-consigo-entrar-na-minha-conta_37a8Qrk-B1kW...


In [12]:
df.shape

(1010, 3)